# Feature selection

In [1]:
import os
import time
from math import log

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import display

t = time.time()

In [2]:
df = pd.read_csv('data/df_train_prepro.csv').sample(frac=1)
df.shape

(1610494, 48)

In [3]:
X = df[['contextid',
        'dayssincelastvisitdouble',
        'campaignctrlast24h',
        'nbdisplay_1hour',
        'nbdayssincelastclick',
        'display_size',
        'ltf_nbpartnerdisplayssincelastclick',
        'ltf_nbglobaldisplay_4w',
        'ltf_nbpartnerclick_90d',
        'ltf_nbpartnerdisplay_90d',
        'ltf_nbpartnersales_90d',
        'nbdisplayglobalapprox_1d_sum_xdevice',
        'zonecostineuro']]
y = df['is_display_clicked']

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

columns_scaled = X.select_dtypes(include=np.number).columns

## Méthode LASSO

On choisit `fit_intercept=False` et `normalize=False` car les données sont déjà centrées et réduites.

In [5]:
def run_LassoCV(alphas_range, X_scaled, y):
    lcv = LassoCV(alphas=alphas_range, normalize=False, fit_intercept=False, random_state=0, n_jobs=-1).fit(X_scaled, y)

    print(f"Best alpha : {lcv.alpha_}")

    lasso = Lasso(fit_intercept=False, normalize=False, alpha=lcv.alpha_)
    lasso.fit(X_scaled, y)
    coeff_df = pd.DataFrame(np.transpose(lasso.coef_), columns_scaled, columns=['Coefficient'])

    display(coeff_df.sort_values(by='Coefficient', ascending=False))

    print(f"\nNombre de coeffs non nuls : {len(coeff_df[coeff_df.Coefficient != 0])}")

In [6]:
%%time
alphas_range = np.linspace(0.001, 1, num=100)
print(alphas_range[:5])
run_LassoCV(alphas_range, X_scaled, y)

[0.001      0.01109091 0.02118182 0.03127273 0.04136364]
Best alpha : 0.001


,Coefficient
zonecostineuro,0.043662
display_size,0.023909
campaignctrlast24h,0.023155
ltf_nbpartnerclick_90d,0.002896
dayssincelastvisitdouble,-0.000000
ltf_nbglobaldisplay_4w,-0.000000
ltf_nbpartnerdisplay_90d,-0.000000
ltf_nbpartnersales_90d,-0.000000
ltf_nbpartnerdisplayssincelastclick,-0.000031
nbdisplayglobalapprox_1d_sum_xdevice,-0.002280



Nombre de coeffs non nuls : 9
Wall time: 8.36 s


## Variance feature selection

In [7]:
from sklearn.feature_selection import VarianceThreshold

In [8]:
def low_variance_feature_selection(X, threshold) :
    sel = VarianceThreshold(threshold)
    sel.fit(X)
    print(sel.get_support(indices=True))

    df = pd.DataFrame(sel.variances_, index=X.columns, columns=['Variance']).sort_values(by='Variance', ascending=False)

    display(df)

    return sel.transform(X)

In [9]:
low_variance_feature_selection(X, 0.01)

[ 0  1  3  4  5  6  7  8  9 10 11 12]


,Variance
display_size,9.444482e+09
ltf_nbglobaldisplay_4w,2.607098e+04
ltf_nbpartnerdisplay_90d,1.061908e+04
nbdisplayglobalapprox_1d_sum_xdevice,4.449996e+03
dayssincelastvisitdouble,2.642414e+03
nbdayssincelastclick,2.066988e+03
ltf_nbpartnerdisplayssincelastclick,7.590539e+02
nbdisplay_1hour,7.176346e+01
zonecostineuro,8.801236e+00
ltf_nbpartnerclick_90d,6.238911e+00


array([[6.00000000e+00, 1.94336000e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 2.40533203e-01],
       [9.00000000e+00, 6.11000000e-01, 1.40000000e+01, ...,
        0.00000000e+00, 5.18815278e+01, 3.40449905e+00],
       [8.00000000e+00, 2.81100000e+00, 3.00000000e+00, ...,
        0.00000000e+00, 3.94738889e+01, 1.86483905e-01],
       ...,
       [3.00000000e+00, 3.95980000e+01, 0.00000000e+00, ...,
        0.00000000e+00, 1.68551111e+02, 1.70969386e-02],
       [3.00000000e+00, 2.31860000e+01, 2.90000000e+01, ...,
        0.00000000e+00, 2.66301111e+02, 1.75312698e-01],
       [6.00000000e+00, 2.50740000e+01, 0.00000000e+00, ...,
        0.00000000e+00, 4.21303704e+01, 9.98989284e-01]])

In [10]:
columns_cat = ['display_env',
            'target_env',
            'campaignscenario',
            'campaignvertical',
            'is_interstitial',
            'device_type']

In [11]:
df_cat = pd.get_dummies(df[columns_cat], columns=columns_cat, drop_first=True)
len(df_cat.columns)

21

In [12]:
Y = df_cat
low_variance_feature_selection(Y, 0.01)

[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20]


,Variance
campaignscenario_13,0.232464
device_type_Desktop,0.216539
campaignvertical_20.0,0.211358
display_env_web,0.207430
campaignscenario_12,0.113986
device_type_iPhone,0.103931
campaignvertical_129.0,0.102221
target_env_2,0.093956
display_env_app_ios,0.082996
campaignscenario_6,0.060119


array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [13]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")

Temps d'exécution total : 00:00:45
